In [ ]:
# CNN for model opti 2 Connect 4

In [ ]:
# Load necessary Packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
import os

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Define a folder path in your Drive where you want the files
#    (Change 'Connect4_Project' to whatever folder name you prefer)
save_path = '/content/drive/My Drive/Connect4_Project'

# 3. Create the folder if it doesn't exist
if not os.path.exists(save_path):
    os.makedirs(save_path)
    print(f"Created new folder: {save_path}")
else:
    print(f"Saving to existing folder: {save_path}")

Mounted at /content/drive
Saving to existing folder: /content/drive/My Drive/Connect4_Project


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ConnectFourWiderCNN(nn.Module):
    def __init__(self):
        super(ConnectFourWiderCNN, self).__init__()

        # Layer 1: 128 Filters (Standard 3x3)
        self.conv1 = nn.Conv2d(2, 128, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(128)

        # Layer 2: 128 Filters (Standard 3x3)
        self.conv2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)

        # Layer 3: 256 Filters + 4x4 Kernel (The "Connect 4" Detector)
        # Padding=2 ensures we don't shrink the board too much
        self.conv3 = nn.Conv2d(128, 256, kernel_size=4, padding=2)
        self.bn3 = nn.BatchNorm2d(256)

        # Layer 4: 256 Filters (Standard 3x3)
        self.conv4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)

        # Fully Connected Layers
        # We need to calculate the flatten size dynamically or trust the math
        # 256 channels * 6 rows * 7 cols (approx due to padding) -> usually larger
        # Let's use a safe large linear layer
        self.fc1 = nn.Linear(256 * 7 * 8, 1024) # Increased to 1024 neurons
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(1024, 7)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))

        # Flatten
        x = x.view(x.size(0), -1)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

print("✅ Wider CNN Architecture Defined")

✅ Wider CNN Architecture Defined


In [ ]:
from torch.utils.data import TensorDataset, DataLoader

### Load already saved trained model checkpath from drive


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConnectFourWiderCNN() # Initialize empty model
model.to(device)

# UPDATE THIS FILENAME to whatever you sent them
file_n = 'checkpoint_epoch_15.pth'
filename = os.path.join(save_path, file_n)
if os.path.exists(filename):
    print(f"Loading {filename}...")
    checkpoint = torch.load(filename, map_location=device)

    # Handle both full checkpoints and simple state_dicts
    if 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
    else:
        model.load_state_dict(checkpoint)

    model.eval() # Set to "Evaluation Mode" (Crucial!)
    print("✅ Model loaded! Ready to play.")
else:
    print(f"❌ Error: Could not find {filename}. Please upload it.")

Loading /content/drive/My Drive/Connect4_Project/checkpoint_epoch_15.pth...
✅ Model loaded! Ready to play.


In [ ]:
import numpy as np
import torch
import sys

# --- 1. Game Logic Functions ---
def create_board():
    return np.zeros((6, 7))

def drop_piece(board, row, col, piece):
    board[row][col] = piece

def is_valid_location(board, col):
    return board[0][col] == 0

def get_next_open_row(board, col):
    for r in range(5, -1, -1):
        if board[r][col] == 0:
            return r

def print_board(board):
    # Print the column headers
    print("\n  0   1   2   3   4   5   6 ")
    print("+---+---+---+---+---+---+---+")

    # Print rows from top (0) to bottom (5)
    # Note: In our matrix, row 0 is usually top, but let's stick to standard printing
    # We iterate 0 to 5.
    for r in range(6):
        row_str = "|"
        for c in range(7):
            if board[r][c] == 1:
                row_str += " X |" # You
            elif board[r][c] == -1:
                row_str += " O |" # AI
            else:
                row_str += "   |" # Empty
        print(row_str)
        print("+---+---+---+---+---+---+---+")
    print()

def winning_move(board, piece):
    # Check horizontal
    for c in range(4):
        for r in range(6):
            if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
                return True
    # Check vertical
    for c in range(7):
        for r in range(3):
            if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
                return True
    # Check positively sloped diagonals
    for c in range(4):
        for r in range(3, 6):
            if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                return True
    # Check negatively sloped diagonals
    for c in range(4):
        for r in range(3):
            if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                return True
    return False

# --- 2. The AI Agent ---
def get_ai_move(board, model, ai_piece):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()

    # Prepare input for CNN (Batch, 2, 6, 7)
    cnn_input = np.zeros((1, 2, 6, 7), dtype=np.float32)
    opponent_piece = 1 if ai_piece == -1 else -1

    # Channel 0 = AI pieces, Channel 1 = Opponent pieces
    cnn_input[0, 0, :, :] = (board == ai_piece)
    cnn_input[0, 1, :, :] = (board == opponent_piece)

    tensor_input = torch.from_numpy(cnn_input).float().to(device)

    with torch.no_grad():
        output = model(tensor_input)

    probabilities = torch.softmax(output, dim=1).cpu().numpy()[0]

    valid_cols = [c for c in range(7) if is_valid_location(board, c)]
    best_col = -1
    best_score = -1.0

    # Pick best valid move
    for col in valid_cols:
        score = probabilities[col]
        if score > best_score:
            best_score = score
            best_col = col

    return best_col



### play game vs cnn

In [ ]:
def play_game():
    board = create_board()
    game_over = False

    # --- CHANGE THIS LINE ---
    turn = 1  # 1 = AI starts first
    # ------------------------

    PLAYER_PIECE = 1
    AI_PIECE = -1

    print("GAME START! You are 'X'. AI is 'O'.")
    print(f"AI (O) goes first!")
    print_board(board)

    while not game_over:
        # --- Human Turn ---
        if turn == 0:
            try:
                col_input = input("Your Turn (Column 0-6): ")
                if col_input.lower() == 'q':
                    break
                col = int(col_input)
                if col < 0 or col > 6:
                    print("Invalid column!")
                    continue
            except ValueError:
                print("Please enter a number!")
                continue

            if is_valid_location(board, col):
                row = get_next_open_row(board, col)
                drop_piece(board, row, col, PLAYER_PIECE)
                print_board(board)

                if winning_move(board, PLAYER_PIECE):
                    print("🎉 YOU WIN! (X)")
                    game_over = True

                turn = 1 # Switch to AI
            else:
                print("Column full!")

        # --- AI Turn ---
        else:
            # AI calculates move
            col = get_ai_move(board, model, AI_PIECE)
            print(f"AI (O) chose column: {col}")

            if is_valid_location(board, col):
                row = get_next_open_row(board, col)
                drop_piece(board, row, col, AI_PIECE)
                print_board(board)

                if winning_move(board, AI_PIECE):
                    print("🤖 AI WINS! (O)")
                    game_over = True

                turn = 0 # Switch to Human
play_game()

GAME START! You are 'X'. AI is 'O'.
AI (O) goes first!

  0   1   2   3   4   5   6 
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+

AI (O) chose column: 3

  0   1   2   3   4   5   6 
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+
|   |   |   | O |   |   |   |
+---+---+---+---+---+---+---+



KeyboardInterrupt: Interrupted by user

In [ ]:
# Optional things to improve model

# Add a 4th Conv2d layer
# Add more data, better quality higher difficulty score
# BatchNorm and Dropout Rate
# Adjust learning rate
# Need to see train loss and accuracy as well as val loss to see what's going on

### CNN vs MCTS

In [ ]:
def check_for_win(board,col):
    # this code is faster than the above code, but it requires knowing where the last checker was dropped
    # it may seem extreme, but in MCTS this function is called more than anything and actually makes up
    # a large portion of total time spent finding a good move.  So every microsecond is worth saving!
    nrow = 6
    ncol = 7
    # take advantage of knowing what column was last played in...need to check way fewer possibilities
    colsum = abs(board[0,col])+abs(board[1,col])+abs(board[2,col])+abs(board[3,col])+abs(board[4,col])+abs(board[5,col])
    row = int(6-colsum)
    if row+3<6:
        vert = board[row,col] + board[row+1,col] + board[row+2,col] + board[row+3,col]
        if vert == 4:
            return 'v-plus'
        elif vert == -4:
            return 'v-minus'
    if col+3<7:
        hor = board[row,col] + board[row,col+1] + board[row,col+2] + board[row,col+3]
        if hor == 4:
            return 'h-plus'
        elif hor == -4:
            return 'h-minus'
    if col-1>=0 and col+2<7:
        hor = board[row,col-1] + board[row,col] + board[row,col+1] + board[row,col+2]
        if hor == 4:
            return 'h-plus'
        elif hor == -4:
            return 'h-minus'
    if col-2>=0 and col+1<7:
        hor = board[row,col-2] + board[row,col-1] + board[row,col] + board[row,col+1]
        if hor == 4:
            return 'h-plus'
        elif hor == -4:
            return 'h-minus'
    if col-3>=0:
        hor = board[row,col-3] + board[row,col-2] + board[row,col-1] + board[row,col]
        if hor == 4:
            return 'h-plus'
        elif hor == -4:
            return 'h-minus'
    if row < 3 and col < 4:
        DR = board[row,col] + board[row+1,col+1] + board[row+2,col+2] + board[row+3,col+3]
        if DR == 4:
            return 'd-plus'
        elif DR == -4:
            return 'd-minus'
    if row-1>=0 and col-1>=0 and row+2<6 and col+2<7:
        DR = board[row-1,col-1] + board[row,col] + board[row+1,col+1] + board[row+2,col+2]
        if DR == 4:
            return 'd-plus'
        elif DR == -4:
            return 'd-minus'
    if row-2>=0 and col-2>=0 and row+1<6 and col+1<7:
        DR = board[row-2,col-2] + board[row-1,col-1] + board[row,col] + board[row+1,col+1]
        if DR == 4:
            return 'd-plus'
        elif DR == -4:
            return 'd-minus'
    if row-3>=0 and col-3>=0:
        DR = board[row-3,col-3] + board[row-2,col-2] + board[row-1,col-1] + board[row,col]
        if DR == 4:
            return 'd-plus'
        elif DR == -4:
            return 'd-minus'
    if row+3<6 and col-3>=0:
        DL = board[row,col] + board[row+1,col-1] + board[row+2,col-2] + board[row+3,col-3]
        if DL == 4:
            return 'd-plus'
        elif DL == -4:
            return 'd-minus'
    if row-1 >= 0 and col+1 < 7 and row+2<6 and col-2>=0:
        DL = board[row-1,col+1] + board[row,col] + board[row+1,col-1] + board[row+2,col-2]
        if DL == 4:
            return 'd-plus'
        elif DL == -4:
            return 'd-minus'
    if row-2 >=0 and col+2<7 and row+1<6 and col-1>=0:
        DL = board[row-2,col+2] + board[row-1,col+1] + board[row,col] + board[row+1,col-1]
        if DL == 4:
            return 'd-plus'
        elif DL == -4:
            return 'd-minus'
    if row-3>=0 and col+3<7:
        DL = board[row-3,col+3] + board[row-2,col+2] + board[row-1,col+1] + board[row,col]
        if DL == 4:
            return 'd-plus'
        elif DL == -4:
            return 'd-minus'
    return 'nobody'

In [ ]:
import numpy as np
import random

def update_board(board_temp, color, column):
    board = board_temp.copy()
    # Calculates the row to drop the piece based on the column sum
    colsum = abs(board[0,column])+abs(board[1,column])+abs(board[2,column])+abs(board[3,column])+abs(board[4,column])+abs(board[5,column])
    row = int(5-colsum)
    if row > -0.5:
        board[row,column] = 1 if color == 'plus' else -1
    return board


def find_legal(board):
    # Identifies columns that are not yet full
    return [i for i in range(7) if abs(board[0,i]) < 0.1]

def look_for_win(board_,color):
    board_ = board_.copy()
    legal = find_legal(board_)
    winner = -1
    for m in legal:
        bt = update_board(board_.copy(),color,m)
        wi = check_for_win(bt,m)
        if wi[2:] == color:
            winner = m
            break
    return winner

def find_all_nonlosers(board,color):
    if color == 'plus':
        opp = 'minus'
    else:
        opp = 'plus'
    legal = find_legal(board)
    poss_boards = [update_board(board,color,l) for l in legal]
    poss_legal = [find_legal(b) for b in poss_boards]
    allowed = []
    for i in range(len(legal)):
        wins = [j for j in poss_legal[i] if check_for_win(update_board(poss_boards[i],opp,j),j) != 'nobody']
        if len(wins) == 0:
            allowed.append(legal[i])
    return allowed
def rollout(board, next_player):
    winner = 'nobody'
    player = next_player
    while winner == 'nobody':
        legal = find_legal(board)
        if len(legal) == 0: return 'tie'
        winning_move = look_for_win(board, player)

        if winning_move != -1:
            move = winning_move
        else:
            # Fallback to random (or your center bias code)
            move = random.choice(legal)
        # ---------------------------------------------

        board = update_board(board, player, move)
        winner = check_for_win(board, move)
        player = 'minus' if player == 'plus' else 'plus'

    return winner

def back_prop(winner,path,color0,md):
    for i in range(len(path)):
        board_temp = path[i]

        md[board_temp][0]+=1
        if winner[2]==color0[0]:
            if i % 2 == 1:
                md[board_temp][1] += 1
            else:
                md[board_temp][1] -= 1
        elif winner[2]=='e': # tie
            # md[board_temp][1] += 0
            pass
        else:
            if i % 2 == 1:
                md[board_temp][1] -= 1
            else:
                md[board_temp][1] += 1




In [ ]:
def mcts(board_temp,color0,nsteps):
    # nsteps is a parameter that determines the skill (and slowness) of the player
    # bigger values of nsteps means the player is better, but also slower to figure out a move.
    board = board_temp.copy()
    ##############################################
    winColumn = look_for_win(board,color0) # check to find a winning column
    if winColumn > -0.5:
        return winColumn # if there is one - play that!
    legal0 = find_all_nonlosers(board,color0) # find all moves that won't immediately lead to your opponent winning
    if len(legal0) == 0: # if you can't block your opponent - just find the 'best' losing move
        legal0 = find_legal(board)
    ##############################################
    # the code above, in between the hash rows, is not part of traditional MCTS
    # but it makes it better and faster - so I included it!
    # MCTS occasionally makes stupid mistakes
    # like not dropping the checker on a winning column, or not blocking an obvious opponent win
    # this avoids a little bit of that stupidity!
    # we could also add this logic to the rest of the MCTS and rollout functions - I just haven't done that yet...
    # feel free to experiment!
    mcts_dict = {tuple(board.ravel()):[0,0]}
    for ijk in range(nsteps):
        color = color0
        winner = 'nobody'
        board_mcts = board.copy()
        path = [tuple(board_mcts.ravel())]
        while winner == 'nobody':
            legal = find_legal(board_mcts)
            if len(legal) == 0:
                winner = 'tie'
                back_prop(winner,path,color0,mcts_dict)
                break
            board_list = []
            for col in legal:
                board_list.append(tuple(update_board(board_mcts,color,col).ravel()))
            for bl in board_list:
                if bl not in mcts_dict.keys():
                    mcts_dict[bl] = [0,0]
            ucb1 = np.zeros(len(legal))
            for i in range(len(legal)):
                num_denom = mcts_dict[board_list[i]]
                if num_denom[0] == 0:
                    ucb1[i] = 10*nsteps
                else:
                    ucb1[i] = num_denom[1]/num_denom[0] + 2*np.sqrt(np.log(mcts_dict[path[-1]][0])/mcts_dict[board_list[i]][0])
            chosen = np.argmax(ucb1)

            board_mcts = update_board(board_mcts,color,legal[chosen])
            path.append(tuple(board_mcts.ravel()))
            winner = check_for_win(board_mcts,legal[chosen])
            if winner[2]==color[0]:
                back_prop(winner,path,color0,mcts_dict)
                break
            if color == 'plus':
                color = 'minus'
            else:
                color = 'plus'
            if mcts_dict[tuple(board_mcts.ravel())][0] == 0:
                winner = rollout(board_mcts,color)
                back_prop(winner,path,color0,mcts_dict)
                break

    maxval = -np.inf
    best_col = -1
    for col in legal0:
        board_temp = tuple(update_board(board,color0,col).ravel())
        num_denom = mcts_dict[board_temp]
        if num_denom[0] == 0:
            compare = -np.inf
        else:
            compare = num_denom[1] / num_denom[0]
        if compare > maxval:
            maxval = compare
            best_col = col
    return (best_col)


In [ ]:
import numpy as np
import random
import os

# --- 1. PERSPECTIVE-AWARE CNN PREPARATION ---
def prepare_for_cnn(board, current_player):
    """
    Converts 6x7 board into 6x7x2 for neural network training.
    Channel 0: 1s where CURRENT player has pieces.
    Channel 1: 1s where OPPONENT has pieces.
    """
    cnn_input = np.zeros((6, 7, 2), dtype=np.float32)

    if current_player == 'plus':
        cnn_input[:, :, 0] = (board == 1).astype(np.float32)
        cnn_input[:, :, 1] = (board == -1).astype(np.float32)
    else:
        # If it's the 'minus' player's turn, flip the perspective
        cnn_input[:, :, 0] = (board == -1).astype(np.float32)
        cnn_input[:, :, 1] = (board == 1).astype(np.float32)

    return cnn_input

import multiprocessing
from multiprocessing import Pool, cpu_count
import random
import numpy as np
from tqdm import tqdm # Optional: for a nice progress bar

# --- WORKER FUNCTION (Runs one game) ---
def play_single_game(args):
    """
    Runs a single game of Connect 4.
    Args matches the tuple passed from the main loop: (game_seed, mcts_skill)
    """
    game_id, mcts_skill = args

    # CRITICAL: Re-seed random number generators for each process
    # preventing all cores from playing the exact same game.
    random.seed(game_id)
    np.random.seed(game_id)

    # --- ORIGINAL LOGIC STARTS HERE ---
    board = np.zeros((6,7))
    player = 'plus'
    last_move = 0

    game_X = []
    game_Y = []

    # Diversity: Play 2-6 random moves first
    for _ in range(random.randint(2, 6)):
        legal = find_legal(board)
        if not legal: break
        last_move = random.choice(legal)
        board = update_board(board, player, last_move)
        if check_for_win(board, last_move) != 'nobody':
            break
        player = 'minus' if player == 'plus' else 'plus'

    # If random moves didn't end the game, start MCTS
    if check_for_win(board, last_move) == 'nobody':
        while True:
            legal = find_legal(board)
            if not legal: break

            # MCTS finds the 'best' move
            best_move = mcts(board, player, mcts_skill)

            # Record Data
            game_X.append(prepare_for_cnn(board, player))
            game_Y.append(best_move)

            # Make Move
            board = update_board(board, player, best_move)
            if check_for_win(board, best_move) != 'nobody':
                break
            player = 'minus' if player == 'plus' else 'plus'

    return game_X, game_Y




In [ ]:
# --- 1. Helper Functions ---
def get_valid_locations(board):
    valid_locations = []
    for col in range(7):
        if is_valid_location(board, col):
            valid_locations.append(col)
    return valid_locations

def check_winner(board, piece):
    return winning_move(board, piece)

In [ ]:

# --- 2. The Battle Arena (Fixed) ---
def battle_arena(model, num_games=10, mcts_sims=150):
    cnn_wins = 0
    mcts_wins = 0
    draws = 0

    print(f"⚔️ STARTING BATTLE: CNN vs. MCTS ({mcts_sims} sims) ⚔️")
    print("-" * 50)

    for i in range(num_games):
        board = create_board()
        game_over = False
        turn = i % 2 # Alternate who starts

        # Track pieces
        if turn == 0:
            cnn_piece = 1
            mcts_piece = -1
            start_msg = "CNN moves first"
        else:
            cnn_piece = -1
            mcts_piece = 1
            start_msg = "MCTS moves first"

        moves_count = 0

        while not game_over:
            # --- CNN TURN ---
            if (turn == 0 and cnn_piece == 1) or (turn == 1 and cnn_piece == -1):
                col = get_ai_move(board, model, cnn_piece)

                if is_valid_location(board, col):
                    row = get_next_open_row(board, col)
                    drop_piece(board, row, col, cnn_piece)

                    if check_winner(board, cnn_piece):
                        cnn_wins += 1
                        game_over = True
                else:
                    print("CNN made invalid move! Forfeit.")
                    mcts_wins += 1
                    game_over = True

            # --- MCTS TURN ---
            else:
                # FIX 1: Convert integer piece to string ('plus'/'minus')
                # Your MCTS function was trained to understand strings, not -1/1
                mcts_color_str = 'plus' if mcts_piece == 1 else 'minus'

                # FIX 2: Use 'nsteps' instead of 'num_simulations'
                col = mcts(board, mcts_color_str, nsteps=mcts_sims)

                if is_valid_location(board, col):
                    row = get_next_open_row(board, col)
                    drop_piece(board, row, col, mcts_piece)

                    if check_winner(board, mcts_piece):
                        mcts_wins += 1
                        game_over = True

            # Check Draw
            if not game_over and len(get_valid_locations(board)) == 0:
                draws += 1
                game_over = True

            # Switch Turn
            turn = 1 - turn
            moves_count += 1

        # Print Result
        if not game_over:
             print(f"Game {i+1}: Error")
        else:
             winner = "CNN" if (check_winner(board, cnn_piece)) else "MCTS"
             if not check_winner(board, cnn_piece) and not check_winner(board, mcts_piece): winner = "Draw"
             print(f"Game {i+1} ({start_msg}): {winner} in {moves_count} moves")

    print("-" * 50)
    print(f"🏆 FINAL RESULTS ({num_games} Games)")
    print(f"CNN Wins: {cnn_wins} ({(cnn_wins/num_games)*100}%)")
    print(f"MCTS Wins: {mcts_wins}")
    print(f"Draws: {draws}")


In [ ]:
battle_arena(model, num_games=10, mcts_sims=600)

⚔️ STARTING BATTLE: CNN vs. MCTS (600 sims) ⚔️
--------------------------------------------------
Game 1 (CNN moves first): MCTS in 42 moves
Game 2 (MCTS moves first): MCTS in 34 moves
Game 3 (CNN moves first): CNN in 27 moves
Game 4 (MCTS moves first): CNN in 29 moves
Game 5 (CNN moves first): CNN in 39 moves
Game 6 (MCTS moves first): MCTS in 34 moves
Game 7 (CNN moves first): CNN in 33 moves
Game 8 (MCTS moves first): CNN in 23 moves
Game 9 (CNN moves first): CNN in 33 moves
Game 10 (MCTS moves first): CNN in 33 moves
--------------------------------------------------
🏆 FINAL RESULTS (10 Games)
CNN Wins: 7 (70.0%)
MCTS Wins: 3
Draws: 0


In [ ]:
battle_arena(model, num_games=20, mcts_sims=800)

⚔️ STARTING BATTLE: CNN vs. MCTS (800 sims) ⚔️
--------------------------------------------------
Game 1 (CNN moves first): MCTS in 32 moves
Game 2 (MCTS moves first): CNN in 21 moves
Game 3 (CNN moves first): CNN in 39 moves
Game 4 (MCTS moves first): MCTS in 40 moves
Game 5 (CNN moves first): CNN in 35 moves
Game 6 (MCTS moves first): MCTS in 34 moves
Game 7 (CNN moves first): CNN in 25 moves
Game 8 (MCTS moves first): CNN in 39 moves
Game 9 (CNN moves first): MCTS in 32 moves
Game 10 (MCTS moves first): MCTS in 40 moves
Game 11 (CNN moves first): CNN in 33 moves
Game 12 (MCTS moves first): CNN in 39 moves
Game 13 (CNN moves first): CNN in 41 moves
Game 14 (MCTS moves first): MCTS in 40 moves
Game 15 (CNN moves first): MCTS in 34 moves
Game 16 (MCTS moves first): MCTS in 42 moves
Game 17 (CNN moves first): MCTS in 24 moves
Game 18 (MCTS moves first): CNN in 31 moves
Game 19 (CNN moves first): MCTS in 40 moves
Game 20 (MCTS moves first): CNN in 27 moves
-------------------------------

In [ ]:
battle_arena(model, num_games=20, mcts_sims=1000)

⚔️ STARTING BATTLE: CNN vs. MCTS (1000 sims) ⚔️
--------------------------------------------------
Game 1 (CNN moves first): MCTS in 36 moves
Game 2 (MCTS moves first): MCTS in 26 moves
Game 3 (CNN moves first): CNN in 37 moves
Game 4 (MCTS moves first): MCTS in 34 moves
Game 5 (CNN moves first): MCTS in 32 moves
Game 6 (MCTS moves first): CNN in 21 moves
Game 7 (CNN moves first): CNN in 41 moves
Game 8 (MCTS moves first): CNN in 27 moves
Game 9 (CNN moves first): MCTS in 40 moves
Game 10 (MCTS moves first): MCTS in 38 moves
Game 11 (CNN moves first): MCTS in 40 moves
Game 12 (MCTS moves first): CNN in 15 moves
Game 13 (CNN moves first): MCTS in 38 moves
Game 14 (MCTS moves first): MCTS in 38 moves
Game 15 (CNN moves first): MCTS in 32 moves
Game 16 (MCTS moves first): MCTS in 38 moves
Game 17 (CNN moves first): CNN in 21 moves
Game 18 (MCTS moves first): CNN in 31 moves
Game 19 (CNN moves first): MCTS in 40 moves
Game 20 (MCTS moves first): MCTS in 24 moves
---------------------------

In [ ]:
battle_arena(model, num_games=20, mcts_sims=1200)

⚔️ STARTING BATTLE: CNN vs. MCTS (1200 sims) ⚔️
--------------------------------------------------
Game 1 (CNN moves first): MCTS in 40 moves
Game 2 (MCTS moves first): CNN in 41 moves
Game 3 (CNN moves first): CNN in 37 moves
Game 4 (MCTS moves first): MCTS in 34 moves
Game 5 (CNN moves first): MCTS in 34 moves
Game 6 (MCTS moves first): MCTS in 32 moves
Game 7 (CNN moves first): MCTS in 42 moves
Game 8 (MCTS moves first): CNN in 27 moves
Game 9 (CNN moves first): MCTS in 34 moves
Game 10 (MCTS moves first): MCTS in 34 moves
Game 11 (CNN moves first): CNN in 25 moves
Game 12 (MCTS moves first): Draw in 42 moves
Game 13 (CNN moves first): MCTS in 36 moves
Game 14 (MCTS moves first): CNN in 33 moves
Game 15 (CNN moves first): CNN in 35 moves
Game 16 (MCTS moves first): CNN in 27 moves
Game 17 (CNN moves first): MCTS in 40 moves
Game 18 (MCTS moves first): MCTS in 34 moves
Game 19 (CNN moves first): MCTS in 24 moves
Game 20 (MCTS moves first): MCTS in 40 moves
---------------------------